In [ ]:
# [E-05] 나랑 닮은 연예인은 누구?

# Step1. 사진 모으기
# Step2. 얼굴 영역 자르기
# Step3. 얼굴 영역의 임베딩 추출하기
# Step4. 모은 연예인들과 비교하기
# Step5. 다양한 재미있는 시각화 시도해 보기
# Step6. 평가 및 회고

# Step1. 사진 모으기

---
 
## 1. 내 사진 찍기

내 사진을 2장 이상 포함합시다.
내 사진들끼리의 얼굴 임베딩 벡터 거리를 구해 봅시다. 닮았음을 확인할 수 있는 좋은 베이스라인이 될 수 있습니다.
혹시 내 사진들끼리의 거리가 멀게 나온다면 왜 그럴지 생각해 봅시다.

## 2. 비교할 연예인 사진 모으기

스터디원 (김성연님, 안가영님)과 본인이 모은 연예인 및 만화캐릭터 사진 (총 176장) 을 기반으로 프로젝트를 진행할 예정입니다.

+ a (원하는 결과값이 나오지 않을 시, https://aiffel.agit.io/g/300307139/wall/324271445 링크에서 참조한 2000여장을 추가할 예정입니다.)

# Step2. 얼굴 영역 자르기

---

임베딩 추출을 위해서는 얼굴 영역을 잘라줘야 합니다.

위 Step1.에서 찾은 이미지들을 이전에 만들었던 get_cropped_face(image_file)을 통해서 잘라줍시다!

자른 이미지는 PIL의 Image.fromarray를 통해서 PIL Image로 변환한 뒤에 저장을 해두면

다양한 재미있는 시각화를 시도해 보는 데 도움이 될 것입니다.

In [1]:
#    힌트
#    from PIL import Image
#    face = get_cropped_face(image_path)
#    pillow_image = Image.fromarray(face)
#    pillow_image.save(path_to_save)

In [6]:
import face_recognition
import os
from PIL import Image

dir_path = os.getenv('HOME')+'/aiffel/face_embedding/Exp05_dataset'
file_list = os.listdir(dir_path)

print(len(file_list))

1


- 해당 프로젝트에서 사용할 이미지들의 총 갯수 (284장)
- os.listdir(): 해당 폴더 내의 모든 파일명과 디렉토리명을 리스트의 형식으로 반환
- 284장의 이미지들의 파일명을 file_list라는 변수에 저장

In [7]:
def get_gropped_face(image_file):
    image = face_recognition.load_image_file(image_file)
    face_locations = face_recognition.face_locations(image)
    a, b, c, d = face_locations[0]
    cropped_face = image[a:c,d:b,:]
    
    return cropped_face

- get_gropped_face(): 이미지 파일에서 얼굴 영역을 가져오는 함수
- face_recognition을 이용하여 이미지 내의 얼굴을 찾기 위해서 Detection 과정을 거치고, 찾은 얼굴 영역을 Crop

In [8]:
def save_cropped_face(dir_path):
    file_list = os.listdir(dir_path)
    
    for file in file_list:
        try:
            img_path = os.path.join(dir_path, file)
            face = get_gropped_face(img_path)
            pillow_image = Image.fromarray(face)
            path_to_save = os.path.join(save_path, file)
            pillow_image.save(path_to_save)
            
        except:
            os.remove(img_path)
            print(file, "was removed")

    print('Cropped images are saved')

save_path = os.getenv('HOME')+'/aiffel/face_embedding/images_cropped'
save_cropped_face(dir_path)

이호진.jpg was removed
Cropped images are saved


- save_cropped_face(): get_gropped_face() 함수를 이용하여 얼굴 영역을 자르고, 해당 얼굴 영역을 이미지로 저장하는 함수
- try, except 구문을 이용하여, 얼굴 인식이 안되는 사진을 삭제 (얼굴 인식이 안되는 이미지가 남아있을 경우, 추후에 문제가 발생하기 때문에 삭제)
- 얼굴 영역을 자른 이미지를 images_cropped 폴더에 저장 (해당 이미지는 마지막 시각화 부분에서 사용할 예정)

In [9]:
import glob

files = glob.glob(os.path.join(save_path, '*.png'))
for file in files:
    if not os.path.isdir(file):
        src = os.path.splitext(file)
        os.rename(file, src[0] + '.jpg')

In [10]:
files = glob.glob(os.path.join(save_path, '*.jpeg'))
for file in files:
    if not os.path.isdir(file):
        src = os.path.splitext(file)
        os.rename(file, src[0] + '.jpg')

- 기존 파일들의 확장자가 jpg, jpeg, png로 다양함
- 마지막에 get_nearest_image() 함수에서 이미지들을 불러올때, '*.jpg'라는 형식을 사용하기 때문에, 모든 이미지 파일의 확장자를 jpg로 변환

# Step3. 얼굴 영역의 임베딩 추출하기

---

이제 얼굴 영역으로 잘린 이미지들이 여럿 생겼을 텐데요.

그 이미지를 face_recognition 을 사용해서 이미지를 불러오고 임베딩을 추출해 봅시다.

다른 사람의 임베딩은 구분할 수 있게 따로 저장해 주세요. 이전에 만들었던 다음 함수들을 활용하면 손쉽게 수행 가능합니다.

- def get_face_embedding(face) : 얼굴영역으로부터 얼굴 임베딩 벡터를 구하는 함수
- def get_face_embedding_dict(dir_path) : 디렉토리 안에 있는 모든 이미지의 임베딩 딕셔너리를 구하는 함수

In [11]:
def get_face_embedding(face):
    return face_recognition.face_encodings(face)

- 얼굴 영역으로부터 얼굴 embedding 벡터를 구하는 함수

In [14]:
def get_face_embedding_dict(dir_path):
    file_list = os.listdir(dir_path)
    embedding_dict = {}
    
    for file in file_list:
        img_path = os.path.join(dir_path, file)
        face = get_gropped_face(img_path)
        embedding = get_face_embedding(face)
        if len(embedding) > 0:  # 얼굴영역 face가 제대로 detect되지 않으면 len(embedding)==0인 경우가 발생하므로 
            # os.path.splitext(file)[0]에는 이미지파일명에서 확장자를 제거한 이름이 담깁니다. 
            embedding_dict[os.path.splitext(file)[0]] = embedding[0]
    
    print("Embedding dictionary was made")
    return embedding_dict

# 내 사진의 임베딩 딕셔너리 출력
embedding_dict = get_face_embedding_dict(dir_path)
embedding_dict['이호진']

Embedding dictionary was made


KeyError: '이호진'

# Step4. 모은 연예인들과 비교하기

모은 연예인 사진들과 자신의 얼굴 사진에서 추출한 임베딩 벡터들 간의

거리를 비교해 보고 가장 작은 연예인이 누구인지 찾아봅시다!

이전 스텝에서 만들었던 get_nearest_face(name, top=5) 함수들을 응용하면 손쉽게 수행 가능합니다.

# Step5. 다양한 재미있는 시각화 시도해 보기

비단 닮은 꼴 연예인을 찾기만 할 것이 아니라,

그들의 얼굴 사진을 활용하여 재미있는 시각화를 시도해 보시기 바랍니다.

tkinter나 flask를 활용해서 닮은꼴을 찾을 이미지를 업로드한 후

그 이미지와 닮은 얼굴을 찾는 형태로 구현해 보는 것도 재미있을 것입니다.

여러분들의 재치가 넘치는 작품들을 기대합니다. 😎